In [ ]:
import sys
sys.path.append('../')

In [ ]:
from kraken import *
from coinbase import *

In [ ]:
exchanges = {'Kraken': {},'Coinbase': {}, 'Coinbase Pro': {}}

In [ ]:
kraken = exchanges['Kraken']

api_key = api_dict['Kraken']['key']
api_sec = api_dict['Kraken']['sec']

kraken['balance'], daily_prices_df, kraken['account_balances'], kraken['currencies'], daily_prices_ls = kraken_pull_all(api_key, api_sec, daily_prices_ls, daily_prices_df)

In [ ]:
coinbase = exchanges['Coinbase']

api_key = api_dict['Coinbase']['key']
api_sec = api_dict['Coinbase']['sec']

coinbase['balance'], daily_prices_df, coinbase['account_balances'], coinbase['currencies'], daily_prices_ls = coinbase_pull_all(api_key, api_sec, daily_prices_ls, daily_prices_df)

In [ ]:
# COMBINE DAILY SALES WITH OUR BALANCE POSITIONS
daily_data = daily_prices_df.merge(kraken['balance'], how='outer', left_index=True, right_index=True)
daily_data = daily_data.merge(coinbase['balance'], how='outer', left_index=True, right_index=True)
daily_data = daily_data.fillna(0)
for asset in list(set(kraken['currencies'] + coinbase['currencies'])):
    if (f'{asset}_x' in daily_data.columns) & (f'{asset}_y' in daily_data.columns):
        daily_data[asset] = daily_data[f"{asset}_x"].astype(float) + daily_data[f"{asset}_y"].astype(float)
        daily_data = daily_data.drop(columns = [f"{asset}_x",f"{asset}_y"])

for asset in list(set(kraken['currencies'] + coinbase['currencies'])):
    daily_data[f'cum_{asset}'] = (daily_data[asset]).cumsum()
    daily_data = daily_data.drop(columns={asset}).rename(columns={f'cum_{asset}':asset})
    
    if asset not in fiat_currencies:
        daily_data[f'{asset}$'] = daily_data[asset]*daily_data[f'{asset}/USD']

In [ ]:
daily_data.tail()

In [ ]:
daily_data.to_hdf('chart.h5','chart')

In [ ]:
import plotly.graph_objects as go
import plotly.colors as pc

df = daily_data[daily_data.index >= '2021-01-20'].copy()
colors = pc.qualitative.Light24
    
fig = go.Figure()
i=0
for asset in list(filter(lambda x:x.endswith("$"), daily_data.columns)) + ['USD']:
    fig.add_trace(go.Scatter(
        x=df.index, 
        y=df[asset],
        line_color='rgba(255,255,255,0)',
        fillcolor=colors[i],
        name=asset.replace('$',''),
        stackgroup='one',
    ))
    i+=1
fig.update_traces(mode='lines')
fig.show()

In [ ]:
import plotly.graph_objects as go
import plotly.colors as pc

df = daily_data[daily_data.index >= '2021-01-20'].copy()
df = daily_data.copy()
colors = pc.qualitative.Light24
i=0
fig = go.Figure()
for asset in ['BTC']:
    fig.add_trace(go.Scatter(
        x=df.index, y=df[x],
        line_color='rgba(255,255,255,0)',
        fillcolor=colors[i],
        name=asset.replace('$',''),
        stackgroup='one',
    ))
    i+=1
fig.update_traces(mode='lines')
fig.show()